#import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install plotly_express
!pip install geopandas
!pip install hdbscan
!pip install hmmlearn
!pip install pomegranate

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from hdbscan import HDBSCAN 
import pickle
from collections import Counter
import plotly_express as px
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")
from pomegranate import *
from hmmlearn import hmm,base
from sklearn import preprocessing
import math

In [ ]:
!ls '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/trips_with_wayID'

trips_with_wayId_1_10.csv  trips_with_wayId_1_27.csv  trips_with_wayId_8_15.csv
trips_with_wayId_1_11.csv  trips_with_wayId_1_28.csv  trips_with_wayId_8_16.csv
trips_with_wayId_1_12.csv  trips_with_wayId_1_29.csv  trips_with_wayId_8_17.csv
trips_with_wayId_1_13.csv  trips_with_wayId_1_2.csv   trips_with_wayId_8_18.csv
trips_with_wayId_1_14.csv  trips_with_wayId_1_30.csv  trips_with_wayId_8_19.csv
trips_with_wayId_1_15.csv  trips_with_wayId_1_31.csv  trips_with_wayId_8_20.csv
trips_with_wayId_1_16.csv  trips_with_wayId_1_3.csv   trips_with_wayId_8_21.csv
trips_with_wayId_1_17.csv  trips_with_wayId_1_4.csv   trips_with_wayId_8_22.csv
trips_with_wayId_1_18.csv  trips_with_wayId_1_5.csv   trips_with_wayId_8_23.csv
trips_with_wayId_1_19.csv  trips_with_wayId_1_6.csv   trips_with_wayId_8_24.csv
trips_with_wayId_1_1.csv   trips_with_wayId_1_7.csv   trips_with_wayId_8_25.csv
trips_with_wayId_1_21.csv  trips_with_wayId_1_8.csv   trips_with_wayId_8_26.csv
trips_with_wayId_1_22.csv  trips_with_wa

##get trip data

In [ ]:
road_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/route_network/clustered'
trips_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/trips_with_wayID'
way_id_on_trips_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.2_wayid_on_trips'

In [ ]:
dfs = []
day_len = []
day_trip_count = []
# days = range(1,32)
days = range(11,12)
# months = range(1,2)
months = range(8,9)

for month in months:
  for day in days:
    try:
      sp_seq_df = pd.read_csv(trips_data_path + f'/trips_with_wayId_{month}_{day}.csv', parse_dates=['time_stamp'])
    except:
      pass

    sp_seq_df = sp_seq_df.sort_values(by=['trip_id', 'time_stamp'])

    trip_count = sp_seq_df.trip_id.value_counts()
    trip_count = trip_count[trip_count >= 12]
    sp_seq_df = sp_seq_df[sp_seq_df.trip_id.isin(trip_count.index)]
    sp_seq_df['month'] = month
    sp_seq_df['day'] = day
    day_len.append(len(sp_seq_df))
    day_trip_count.append(len(trip_count))
    dfs.append(sp_seq_df)
print(day_len)
print(day_trip_count)
df = pd.concat(dfs, ignore_index=True)
df

# df = df[['trip_id', 'way_id', 'day', 'month','route_cluster']]
df.head()

[591084]
[7514]


,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id,way_id,dist,month,day
0,2019-08-11 00:04:31,060000500000000011561716116,14.16137,102.66467,0,7.0,0,461238098,0.153034,8,11
1,2019-08-11 00:04:36,060000500000000011561716116,14.16137,102.66467,0,7.0,0,461238098,0.153034,8,11
2,2019-08-11 00:04:41,060000500000000011561716116,14.16137,102.66467,0,7.0,0,461238098,0.153034,8,11
3,2019-08-11 00:04:46,060000500000000011561716116,14.16137,102.66467,0,7.0,0,461238098,0.153034,8,11
4,2019-08-11 00:04:51,060000500000000011561716116,14.16137,102.66467,0,7.0,0,461238098,0.153034,8,11


##get pickel

In [ ]:
from hdbscan import HDBSCAN 
import pickle
train_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/model/3.2'
pkl_filename = train_path+"/HDB_test_way_eps0.02.pkl"
with open(pkl_filename, 'rb') as file:
    clustering = pickle.load(file)
clustering.labels_

array([164, 193,  -1, ..., 707, 707, 707])

## get way

In [ ]:
way_segment_labeled_df = pd.read_csv(way_id_on_trips_path + f'/lebeled/HDB_label_esp0.02.csv')
way_segment_labeled_df['road_seg'] = way_segment_labeled_df['road_seg'].map(lambda x: eval(x))
way_segment_labeled_df

,way_id,road_seg,mid_lat,mid_lon,count,label
0,686967969,"(35, 449)",17.478832,102.795843,12,164
1,686957536,"(2895, 4520)",17.535332,102.761058,1,193
2,686912257,"(4519, 4520)",17.544479,102.734002,3,-1
3,686911019,"(4516, 4519)",17.556765,102.700705,3,149
4,686910206,"(4516, 4517)",17.566924,102.670093,6,149
...,...,...,...,...,...,...
56106,123076908,"(439, 442)",16.631444,102.800829,84,707
56107,123076908,"(414, 439)",16.616631,102.805282,129,707
56108,123076908,"(414, 562)",16.607361,102.812563,129,707
56109,91540198,"(419, 420)",16.678062,102.804382,49,707


# get rid

In [ ]:
map_way_label = []
for way_id in df['way_id'].to_list():
  label_count = way_segment_labeled_df[way_segment_labeled_df.way_id==way_id][['label', 'count']].sort_values(by='count')
  if len(label_count) != 0:
      map_way_label.append([way_id, label_count.iloc[0].label])
  else:
    map_way_label.append([way_id, -1])
map_way_label = pd.DataFrame(map_way_label, columns=['way_id', 'label'])
map_way_label

,way_id,label
0,461238098,206
1,461238098,206
2,461238098,206
3,461238098,206
4,461238098,206
...,...,...
591079,478988251,96
591080,478988251,96
591081,478988251,96
591082,478988251,96


In [ ]:
# map_way_label = pd.read_csv('/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.2_wayid_on_trips/lebeled/map_way_lable.csv')
# map_way_label

In [ ]:
df_labeled = df
# df_labeled = df_labeled.merge(map_way_label, left_on='way_id', right_on='way_id', how='left')
df_labeled['label'] = map_way_label['label']
df_labeled

,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id,way_id,dist,month,day,label
0,2019-08-11 00:04:31,060000500000000011561716116,14.161370,102.664670,0,7.0,0,461238098,0.153034,8,11,206
1,2019-08-11 00:04:36,060000500000000011561716116,14.161370,102.664670,0,7.0,0,461238098,0.153034,8,11,206
2,2019-08-11 00:04:41,060000500000000011561716116,14.161370,102.664670,0,7.0,0,461238098,0.153034,8,11,206
3,2019-08-11 00:04:46,060000500000000011561716116,14.161370,102.664670,0,7.0,0,461238098,0.153034,8,11,206
4,2019-08-11 00:04:51,060000500000000011561716116,14.161370,102.664670,0,7.0,0,461238098,0.153034,8,11,206
...,...,...,...,...,...,...,...,...,...,...,...,...
591079,2019-08-11 09:33:38,022000200000000000020082551,16.906278,101.914598,58,7.0,7513,478988251,0.005665,8,11,96
591080,2019-08-11 09:36:38,022000200000000000020082551,16.927677,101.925420,63,7.0,7513,478988251,0.028583,8,11,96
591081,2019-08-11 09:39:38,022000200000000000020082551,16.956223,101.924398,56,7.0,7513,478988251,0.023415,8,11,96
591082,2019-08-11 09:42:38,022000200000000000020082551,16.977960,101.911260,62,7.0,7513,478988251,0.017872,8,11,96


#func score

In [ ]:
def score_hmmlearn(hmm,seq,label,encode):
  predict = encode.inverse_transform(hmm.predict(seq))
  tp = 0
  for _state_name, _label in zip(predict, label):
    if _state_name == _label:
      tp+=1
  total = len(label)
  return tp/total

In [ ]:
def score(model, seq, label):
  p = []
  prob,path = model.viterbi(seq)
  for s in path[1:]:
    p.append(s[1].name)
  tp = 0
  for _state_name, _label in zip(p, label):
    if _state_name == _label:
      tp+=1
  total = len(label)
  return tp/total

In [ ]:
def for_model_fit(x_train):
  x = []
  length = []
  for seq  in x_train:
    length.append(len(seq))
    for i in seq:
      x.append(i)

  x = np.array(x)
  x = x.reshape(len(x),1)
  x = x.astype(int)
  return x,length

#hmmlearn

In [ ]:
state_names = []
uniq_label = df_labeled.label.unique()
for label in uniq_label:
  state_names.append(str(label))
state_names[:5]

X = []
labels = []
trip_df = df_labeled
for month in months:
  for day in days:
    day_df = trip_df[(df.day == day) & (trip_df.month == month)]
    uniq_trips = day_df.trip_id.unique()
    for trip_id in uniq_trips:
      trip = day_df[day_df.trip_id==trip_id]
      X.append(trip.way_id.to_numpy().astype(str))
      # labels.append(['None-start']+[str(l) for l in trip.route_cluster.to_list()])
      labels.append([str(l) for l in trip.label.to_list()])
print(X[0][:5])
print(labels[0][:5])
print(len(X))
print(len(labels))

['461238098' '461238098' '461238098' '461238098' '461238098']
['206', '206', '206', '206', '206']
7514
7514


In [ ]:
import random
random.seed(42)

X2 = X.copy()
labels2 = labels.copy()

temp = list(zip(X2, labels2))
random.shuffle(temp)
X2, labels2 = zip(*temp)
X2 = list(X2)
labels2 = list(labels2)

print(X[0][:5])
print(X2[0][:5])

print(labels[0][:5])
print(labels2[0][:5])

['461238098' '461238098' '461238098' '461238098' '461238098']
['91844882' '119551821' '161160546' '161160546' '629818891']
['206', '206', '206', '206', '206']
['270', '270', '-1', '-1', '269']


In [ ]:
way_seq = df_labeled.way_id.to_list()
not_stop_mask = [True]+[(a != b) for a, b in zip(way_seq[:-1], way_seq[1:])]
df_labeled = df_labeled.loc[not_stop_mask]

way_df = df_labeled[['way_id', 'lat', 'lon']]
way_df = way_df.drop_duplicates(subset='way_id')

sp_list = set(map(str, way_df['way_id'].tolist()))

In [ ]:
# total = len(X2)
# fold = int(total/10)
# start = 0
# starts = [] # [0, 952, 1904, 2856, 3808, 4760, 5712, 6664, 7616, 8568]
# for i in range(10):
#   starts.append(start)
#   start+=fold

##for pomegranate

In [ ]:
accuracy_train = []
accuracy_test = []
X_train = X2
labels_train = labels2
uniq_sp = set()
for seq in X_train:
  for sp in seq:
    uniq_sp.add(sp)
uniq_label = set()

for label in labels_train:
  for uniq in list(set(label)):
    uniq_label.add(uniq)

unseen_sp = sp_list - uniq_sp
print(len(unseen_sp))
X_train.append(np.array(list(unseen_sp)))
labels_train.append(['-1'] * len(unseen_sp))

model = HiddenMarkovModel.from_samples(
    DiscreteDistribution, 
    n_components=len(uniq_label), 
    X=X_train, 
    labels=labels_train,
    algorithm='labeled',
    state_names=list(uniq_label), 
    inertia=0.001,
    max_iterations=10,
    n_jobs=-1
)


0


In [ ]:
save_model = model.to_json()

In [ ]:
import json
with open('/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.2_wayid_on_trips/model_1_1.json','w') as f:
  json.dump(save_model,f)

In [ ]:
X_test = X2
labels_test = labels2

In [ ]:
accuracy_train = []
accuracy_test = []
fail_count = 0
accuracy_train_percent = []
for percent in [0.25, 0.5, 0.75, 0.9]:
  accu = 0
  for seq, label in zip(X_train, labels_train):
    sc = score(model, np.array(seq[:int(len(seq)*percent)]), label[1:])
    if sc > 0:
      accu += sc
      # accu += 1
  train_score = accu/len(X_train)
  accuracy_train_percent.append(train_score)
accuracy_train.append(accuracy_train_percent)

accuracy_test_percent = []
for percent in [0.25, 0.5, 0.75, 0.9]:
  accu = 0
  for seq, label in zip(X_test, labels_test):
    try:
      sc = score(model, np.array(seq[:int(len(seq)*percent)]), label[1:])
    except:
      print("Fail!!", end="\t")
      fail_count = fail_count + 1
      continue
    
    if sc > 0:
      accu += sc
      # accu += 1
  test_score = accu/len(X_test)
  accuracy_test_percent.append(test_score)
accuracy_test.append(accuracy_test_percent)


print(f'Train score')
print(f'\t25% trip traverse : {accuracy_train_percent[0]}')
print(f'\t50% trip traverse : {accuracy_train_percent[1]}')
print(f'\t75% trip traverse : {accuracy_train_percent[2]}')
print(f'\t90% trip traverse : {accuracy_train_percent[3]}')
print(f'Test score')
print(f'\t25% trip traverse : {accuracy_test_percent[0]}')
print(f'\t50% trip traverse : {accuracy_test_percent[1]}')
print(f'\t75% trip traverse : {accuracy_test_percent[2]}')
print(f'\t90% trip traverse : {accuracy_test_percent[3]}')
print('fail seq = ',fail_count)

Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!

##for hmmlearn

In [ ]:
accuracy_train = []
accuracy_test = []
 
for start in starts: # 10
  X_train = X2[:start]+X2[start+fold:]
  X_test = X2[start:start+fold]
  labels_train = labels2[:start]+labels2[start+fold:]
  labels_test = labels2[start:start+fold]

    # บรรทัดนี้ปลอดภัยไว้ก่อน
  uniq_sp = set()
  for seq in X_train:
    for sp in seq:
      uniq_sp.add(sp)
  uniq_label = set()
  # for label in labels_train:
  for label in labels_train:
    for uniq in list(set(label)):
      uniq_label.add(uniq)
  
  unseen_sp = sp_list - uniq_sp
  print(len(unseen_sp))
  X_train.append(np.array(list(unseen_sp)))
  labels_train.append(['-1'] * len(unseen_sp))

  print(uniq_label)
  uniq_label_for_encode = []
  for i  in uniq_label:
    uniq_label_for_encode.append(i)

  encode = preprocessing.LabelEncoder()
  encode.fit(uniq_label_for_encode)

  X_train_for_fit,length = for_model_fit(X_train)

  model = hmm.MultinomialHMM(n_components=len(uniq_label),n_iter=100,random_state=42)

  model.fit(X_train_for_fit,length)

#   accuracy_train_percent = []
#   for percent in [0.25, 0.5, 0.75, 0.9]:
#     accu = 0
#     for seq, label in zip(X_train, labels_train):
#       sc = score(model, np.array(seq[:int(len(seq)*percent)]), labels,encode)
#       if sc > 0:
#         # accu += sc
#         accu += 1
#     train_score = accu/len(X_train)
#     accuracy_train_percent.append(train_score)
#   accuracy_train.append(accuracy_train_percent)

#   accuracy_test_percent = []
#   for percent in [0.25, 0.5, 0.75, 0.9]:
#     accu = 0
#     for seq, label in zip(X_test, labels_test):
#       sc = score(model, np.array(seq[:int(len(seq)*percent)]), labels,encode)
#       if sc > 0:
#         # accu += sc
#         accu += 1
#     test_score = accu/len(X_test)
#     accuracy_test_percent.append(test_score)
#   accuracy_test.append(accuracy_test_percent)

#   print(f'Score {start} - {start+fold}')
#   print(f'Train score')
#   print(f'\t25% trip traverse : {accuracy_train_percent[0]}')
#   print(f'\t50% trip traverse : {accuracy_train_percent[1]}')
#   print(f'\t75% trip traverse : {accuracy_train_percent[2]}')
#   print(f'\t90% trip traverse : {accuracy_train_percent[3]}')
#   print(f'Test score')
#   print(f'\t25% trip traverse : {accuracy_test_percent[0]}')
#   print(f'\t50% trip traverse : {accuracy_test_percent[1]}')
#   print(f'\t75% trip traverse : {accuracy_test_percent[2]}')
#   print(f'\t90% trip traverse : {accuracy_test_percent[3]}')

# print('all score')
# print('train score: ')
# for i in range(4):
#   sum = 0
#   for fold in accuracy_train:
#     sum += fold[i]
#   print(f'\ttrip traverse {i} : {sum/10}')

# print('test score: ')
# for i in range(4):
#   sum = 0
#   for fold in accuracy_test:
#     sum += fold[i]
#   print(f'\ttrip traverse {i} : {sum/10}')

Fitting a model with 437772447949 free scalar parameters with only 11561 data points will result in a degenerate solution.


53
{'101', '65', '254', '435', '115', '312', '686', '447', '661', '353', '701', '393', '249', '675', '601', '217', '39', '432', '448', '292', '666', '160', '1', '494', '583', '281', '560', '155', '332', '243', '549', '186', '488', '44', '22', '628', '515', '72', '572', '674', '326', '602', '228', '480', '264', '119', '168', '313', '321', '170', '250', '585', '348', '272', '205', '51', '658', '706', '110', '15', '251', '55', '279', '88', '85', '657', '192', '470', '304', '495', '247', '59', '665', '294', '130', '452', '201', '52', '381', '127', '121', '482', '561', '255', '481', '531', '143', '46', '295', '513', '230', '394', '597', '391', '446', '653', '492', '576', '650', '679', '505', '694', '340', '434', '27', '86', '490', '664', '163', '570', '415', '26', '131', '682', '438', '288', '57', '226', '539', '165', '565', '499', '659', '270', '636', '103', '29', '308', '656', '563', '689', '529', '145', '397', '266', '318', '497', '97', '643', '234', '273', '104', '194', '355', '8', '199

In [ ]:
for start in starts: # 10
  X_train = X2[:start]+X2[start+fold:]
  x,L = for_model_fit(X_train)
  print(len(x))

11508
11090
11431
11364
11575
11292
11362
11181
11338
11435


In [ ]:
11508*11508

132434064

In [ ]:
for start in starts: # 10
  X_train = X2[:start]+X2[start+fold:]
  print(len(X_train))

321
321
321
321
321
321
321
321
321
321


In [ ]:
X_train = X2[:0]+X2[0+fold:]

In [ ]:
len(X_train)

321

In [ ]:
x,L = for_model_fit(X_train)

In [ ]:
len(x)

11508

In [ ]:
len(L)

321

In [ ]:
model = hmm.MultinomialHMM(n_components=len(uniq_label),n_iter=1000,random_state=42)
model.fit(x,L)

Fitting a model with 446626276825 free scalar parameters with only 11508 data points will result in a degenerate solution.
